# Conway's Game of Life
This notebook demostrates how you can develop interactive notebooks with lgo by developping [Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life)

In [1]:
// 게임오브 라이브 구조체
type GameOfLife struct {
    // 현재
    cur [][]bool
    // 버프
    buf [][]bool
    // 세대
    generation int
}

// 새 게임오브 라이프 함수
func NewGameOfLife(w, h int) *GameOfLife {
    if w < 3 || h < 3 {
        panic("too small")
    }
    // 
    cur := make([][]bool, w)
    buf := make([][]bool, w)
    for i := 0; i < w; i++ {
        cur[i] = make([]bool, h)
        buf[i] = make([]bool, h)
    }
    return &GameOfLife{
        cur: cur,
        buf: buf,
    }
}

func (g *GameOfLife) nextPixel(x, y int) bool {
    c := 0
    for i := x-1; i < x+2; i++ {
        for j := y-1; j < y+2; j++ {
            if (i != x || j != y) && g.cur[i][j] {
                c += 1
            }
        }
    }
    if !g.cur[x][y] {
        // dead
        return c == 3
    }
    if c == 2 || c == 3 {
        return true
    }
    return false
}

func (g *GameOfLife) next() {
    w := len(g.cur)
    h := len(g.cur[0])
    for i := 0; i < w; i++ {
        for j := 0; j < h; j++ {
            if i == 0 || i == w-1 || j == 0 || j == h-1 {
                // border
                g.buf[i][j] = false
                continue
            }
            g.buf[i][j] = g.nextPixel(i, j)
        }
    }
    tmp := g.cur
    g.cur = g.buf
    g.buf = tmp
    g.generation++
}

func (g *GameOfLife) Set(x, y int) {
    if x >= 1 && y >= 1 && x < len(g.cur)-1 && y < len(g.cur[0])-1 {
        g.cur[x][y] = true
    }
}

func (g *GameOfLife) Size() (int, int) {
    return len(g.cur), len(g.cur[0])
}

In [3]:
import (
    "bytes"
    "fmt"
    "math/rand"
    "time"
    "os"
)

// Canvas renders the content of GameOfLife to HTML Canvas.
type Canvas struct {
    id string
    jsid string
    width int
    height int
    g *GameOfLife
}

func NewCanvas(game *GameOfLife, width, height int) *Canvas {
    return &Canvas{
        id: fmt.Sprintf("canvas%d", rand.Int63()),
        width: width,
        height: height,
        g: game,
    }
} 

func (c *Canvas) displayCanvas() {
    _ctx.Display.HTML(fmt.Sprintf(
        `<b id="%s-label"></b><br><canvas id="%s" width="%d" height="%d"></canvas>`,
        c.id, c.id, c.width, c.height), nil)
    c.jsid = ""
}

func (c *Canvas) renderGame() {
    g := c.g
    w := len(g.cur)
    h := len(g.cur[0])
    px := float32(c.width)/float32(w)
    py := float32(c.height)/float32(h)
    
    var b bytes.Buffer
    b.WriteString(fmt.Sprintf(`
(function(){
var canvasID = "%s";
var width = %d;
var height = %d;
`, c.id, c.width, c.height))
    b.WriteString(fmt.Sprintf(`
document.getElementById(canvasID + '-label').innerHTML = 'Generation: %d';
`, g.generation))
    b.WriteString(`
var canvas = document.getElementById(canvasID);
var ctx = canvas.getContext('2d');
ctx.clearRect(0, 0, width, height);
`)
    
    for x := 0; x < w; x++ {
        for y := 0; y < h; y++ {
            if g.cur[x][y] {
                b.WriteString(fmt.Sprintf(
                    "ctx.fillRect(%f, %f, %f, %f);\n",
                    px * float32(x), py * float32(y), px, py))
            }
        }
    }
    b.WriteString(`})()`)
    _ctx.Display.JavaScript(b.String(), &c.jsid)
}

func (c *Canvas) DisplayAnimation(step int, interval time.Duration) {
    if interval < 10 * time.Millisecond {
        fmt.Fprintf(os.Stderr, "interval is too small: %v", interval)
        return
    }
    c.displayCanvas()
    c.renderGame()
    prev := time.Now()
    for i := 0; step < 0 || i < step; i++ {
        c.g.next()
        time.Sleep(interval-time.Now().Sub(prev))
        prev = time.Now()
        c.renderGame()
    }
}

# Oscillators

In [4]:
{
    g := NewGameOfLife(20, 10)
    c := NewCanvas(g, 400, 200)
    
    var x, y int
    x, y = 1, 1
    g.Set(x, y+1)
    g.Set(x+1, y+1)
    g.Set(x+2, y+1)
    
    x, y = 5, 1
    g.Set(x+1, y+1)
    g.Set(x+2, y+1)
    g.Set(x+3, y+1)
    g.Set(x, y+2)
    g.Set(x+1, y+2)
    g.Set(x+2, y+2)
    
    x, y = 11, 1
    g.Set(x, y)
    g.Set(x+1, y)
    g.Set(x, y+1)
    g.Set(x+1, y+1)
    g.Set(x+2, y+2)
    g.Set(x+3, y+2)
    g.Set(x+2, y+3)
    g.Set(x+3, y+3)
    
    c.DisplayAnimation(20, 250*time.Millisecond)
}

# Gliders

In [5]:
func leftRotate(g *GameOfLife) *GameOfLife {
    w, h := g.Size()
    n := NewGameOfLife(h, w)
    n.generation = g.generation
    for i := 0; i < w; i++ {
        for j := 0; j < h; j++ {
            n.cur[j][h-1-i] = g.cur[i][j]
        }
    }
    return n
}

func addGlider(g *GameOfLife, x, y int) {
    g.Set(x, y+2)
    g.Set(x+1, y)
    g.Set(x+1, y+2)
    g.Set(x+2, y+1)
    g.Set(x+2, y+2)
}

{   
    g := NewGameOfLife(160, 160)
    for r := 0; r < 4; r++ {
        max := 7
        for i := 0; i < max; i++ {
            for j := 0; j < max; j++ {
                if i + j >= max {
                    continue
                }
                addGlider(g, i*10+5, j*10+5)
            }   
        }
        g = leftRotate(g)
    }
    c := NewCanvas(g, 480, 480)
    c.DisplayAnimation(300, 100*time.Millisecond)
}

main routine canceled


# Random

In [6]:
import (
    "math/rand"
)

{   
    w, h := 200, 200
    g := NewGameOfLife(w, h)
    c := NewCanvas(g, 480, 480)
    for i := 1; i < w; i++ {
        for j := 1; j < h; j++ {
            if rand.Int()%2!=0 {
                g.Set(i, j)
            }
        }
    }
    
    for i := 0; i < 10; i++ {
        for j := 0; j < 10; j++ {
            addGlider(g, i*8, j*9)
        }
    }
    c.DisplayAnimation(1000, 100*time.Millisecond)
}

main routine canceled
